In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
# from datasets import load_dataset
# import numpy as np
# import torch

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# paraphrase_tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
# paraphrase_model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# task_tokenizer = AutoTokenizer.from_pretrained("nateraw/bert-base-uncased-ag-news")
# task_model = AutoModelForSequenceClassification.from_pretrained("nateraw/bert-base-uncased-ag-news").to(device).eval()

# ag_news = load_dataset("ag_news", split="test[:10%]")

In [ ]:
# def paraphrase(
#     question,
#     num_beams=5,
#     num_beam_groups=5,
#     num_return_sequences=5,
#     repetition_penalty=10.0,
#     diversity_penalty=3.0,
#     no_repeat_ngram_size=2,
#     temperature=0.7,
#     max_length=128
# ):
#     input_ids = paraphrase_tokenizer(
#         f'paraphrase: {question}',
#         return_tensors="pt", padding="longest",
#         max_length=max_length,
#         truncation=True,
#     ).input_ids.to(device)
    
#     outputs = paraphrase_model.generate(
#         input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
#         num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
#         num_beams=num_beams, num_beam_groups=num_beam_groups,
#         max_length=max_length, diversity_penalty=diversity_penalty
#     )

#     res = paraphrase_tokenizer.batch_decode(outputs, skip_special_tokens=True)

#     return res

# input_text = ag_news[0]["text"]
# print(f"Input text: {input_text}")

# paraphrases = paraphrase(input_text)
# paraphrases

In [ ]:
# with torch.no_grad():
#     input_texts = [input_text] + paraphrases
#     inputs = task_tokenizer(input_texts, return_tensors="pt", padding="longest").to(device)
#     outputs = task_model(**inputs)
#     predictions = torch.argmax(outputs.logits, dim=-1)
#     probs = outputs.logits.softmax(dim=1).cpu().numpy()

# probs

In [ ]:
# def calculate_entropy(current_probs):
#     return -np.sum(current_probs * np.log(current_probs))

# entropies = [calculate_entropy(class_dist) for class_dist in probs]
# display(entropies)

# mean_distribution = np.mean(probs, axis=0).tolist()
# display(mean_distribution)

# mean_distribution_entropy = calculate_entropy(mean_distribution)
# display(mean_distribution_entropy)


# Model Updates

In [46]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from sklearn.metrics import classification_report
from datasets import load_dataset, Dataset
from torch.optim import AdamW
from tqdm import tqdm
import plotly.express as px
import pandas as pd
import numpy as np
import torch

In [47]:
ag_news = load_dataset("ag_news", split="test[:20]")
ag_news = pd.read_csv("/home/kyle/repos/Parameter-Free-LM-Editing/datasets/ag_news_twitter/shifted_test_set_gpt3.csv")
ag_news.rename(columns={"tweet summary": "text"}, inplace=True)
display(ag_news.head())
ag_news = Dataset.from_pandas(ag_news.sample(50))

Found cached dataset ag_news (/home/kyle/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


,article summary,label,text,prompt
0,Fears for T N pension after talks Unions repre...,2,Unions express disappointment in talks with Fe...,V1
1,The Race is On: Second Private Team Sets Launc...,3,🚀👨‍🚀 #SpaceRace update: Second private team to...,V1
2,Ky. Company Wins Grant to Study Peptides (AP) ...,3,"""Chemistry researcher's Ky. startup wins grant...",V1
3,Prediction Unit Helps Forecast Wildfires (AP) ...,3,"🔥💨🌲Prediction Unit helps forecast wildfires, s...",V1
4,Calif. Aims to Limit Farm-Related Smog (AP) AP...,3,"""California takes on smog w emissions rules fo...",V1


In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
paraphrase_tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
paraphrase_model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device).eval()

task_tokenizer = AutoTokenizer.from_pretrained("nateraw/bert-base-uncased-ag-news")
task_model = AutoModelForSequenceClassification.from_pretrained("nateraw/bert-base-uncased-ag-news").to(device)

In [49]:
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = paraphrase_tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = paraphrase_model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = paraphrase_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

In [50]:
baseline_judgments = []
for index in tqdm(range(len(ag_news))):
    with torch.no_grad():
        task_model.eval()
        current_input = ag_news[index]["text"]
        input_tokens = task_tokenizer(current_input, return_tensors="pt", padding="longest").to(device)
        input_logits = task_model(**input_tokens).logits
        judgment = torch.argmax(input_logits)
        baseline_judgments.append(judgment.detach().item())

100%|██████████| 50/50 [00:00<00:00, 106.97it/s]


In [51]:

optimizer = AdamW(task_model.parameters(), lr=0.000003)
optimizer.zero_grad()
task_model.train()

memo_judgments = []
entropies = []
for index in tqdm(range(len(ag_news))):
    task_model.train()
    optimizer.zero_grad()
    
    current_input = ag_news[index]["text"]
    augmentations = paraphrase(current_input)
    
    aug_tokens = task_tokenizer(augmentations, return_tensors="pt", padding="longest").to(device)
    aug_logits = task_model(**aug_tokens).logits
    aug_probs = aug_logits.softmax(dim=1)
    marginal_probs = aug_probs.mean(dim=0)
    marginal_entropy = -torch.sum(marginal_probs * torch.log(marginal_probs))
    marginal_entropy.backward()
    entropies.append(marginal_entropy.item())
    optimizer.step()
    
    # print(augmentations)
    print(f"Entropy: {marginal_entropy.item()}")
    # print(aug_probs)
    # print(marginal_probs)
    
    with torch.no_grad():
        task_model.eval()
        input_tokens = task_tokenizer(current_input, return_tensors="pt").to(device)
        input_logits = task_model(**input_tokens).logits
        judgment = torch.argmax(input_logits)
        memo_judgments.append(judgment.detach().item())
        
    # break

  2%|▏         | 1/50 [00:01<00:57,  1.17s/it]

Entropy: 0.6247378587722778


  4%|▍         | 2/50 [00:02<00:57,  1.21s/it]

Entropy: 0.1710158735513687


  6%|▌         | 3/50 [00:03<00:52,  1.12s/it]

...

Entropy: 0.020216278731822968


 98%|█████████▊| 49/50 [01:01<00:01,  1.32s/it]

Entropy: 0.04557943716645241


100%|██████████| 50/50 [01:02<00:00,  1.25s/it]

Entropy: 0.9055253267288208


In [52]:
px.line(x=np.arange(len(ag_news)), y=entropies)

In [54]:
print(classification_report(ag_news["label"], baseline_judgments))
print(classification_report(ag_news["label"], memo_judgments))

              precision    recall  f1-score   support

           0       1.00      0.70      0.82        10
           1       0.94      1.00      0.97        16
           2       1.00      0.67      0.80        15
           3       0.56      1.00      0.72         9

    accuracy                           0.84        50
   macro avg       0.88      0.84      0.83        50
weighted avg       0.90      0.84      0.84        50

              precision    recall  f1-score   support

           0       1.00      0.70      0.82        10
           1       0.94      1.00      0.97        16
           2       1.00      0.80      0.89        15
           3       0.64      1.00      0.78         9

    accuracy                           0.88        50
   macro avg       0.90      0.88      0.87        50
weighted avg       0.92      0.88      0.88        50

